### Import packages

In [1]:
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition

### Create model

In [2]:
# Define model
m = ConcreteModel()

# Define sets
m.ITEMS = Set(initialize=['A', 'B', 'C'])
m.WEEKS = Set(initialize=range(1,5))

# Define Decision Variable
m.MarkPct = Var(m.ITEMS, m.WEEKS, domain = NonNegativeReals, bounds = (0,1))

# Define Implicit Variables
m.MarkPrice = Var(m.ITEMS, m.WEEKS, domain = NonNegativeReals)
m.Demand = Var(m.ITEMS, m.WEEKS, domain = NonNegativeReals)
m.Revenue = Var(m.ITEMS, m.WEEKS, domain = NonNegativeReals)
m.FinalInv = Var(m.ITEMS, m.WEEKS, domain = NonNegativeReals)

# Define Params
m.basePrice = Param(m.ITEMS, initialize = {'A':21.5, 'B':29.5, 'C': 30.5})
m.initInv = Param(m.ITEMS, initialize = {'A':10, 'B':30, 'C':41})
m.maxPct = 0.80
m.cap = 20
m.dispCost = 1.2

# Define equations for implicit variables

# Define Markdown Price
def Define_MarkPrice_Rule(m,i,w):
    return (1-m.MarkPct[i,w])*m.basePrice[i] == m.MarkPrice[i,w]
m.Define_MarkPrice =  Constraint(m.ITEMS, m.WEEKS, rule = Define_MarkPrice_Rule)

# Define Demand
def Define_Demand_Rule(m,i,w):
    return  m.MarkPct[i,w]*m.initInv[i] == m.Demand[i,w]
m.Define_Demand =  Constraint(m.ITEMS, m.WEEKS, rule = Define_Demand_Rule)

# Define Revenue
def Define_Revenue_Rule(m,i,w):
    return m.MarkPrice[i,w]*m.Demand[i,w] == m.Revenue[i,w]
m.Define_Revenue =  Constraint(m.ITEMS, m.WEEKS, rule = Define_Revenue_Rule)

# Define Final Inv
def Define_FinalInv_Rule(m,i,w):
    if w==1:
        return m.FinalInv[i,w] == m.initInv[i] - m.Demand[i,w]
    else:
        return m.FinalInv[i,w] == m.FinalInv[i,w-1] - m.Demand[i,w]
m.Define_FinalInv =  Constraint(m.ITEMS, m.WEEKS, rule = Define_FinalInv_Rule)


# Add limitations on how soon to markdown and how much (made up in class)
m.maxPctW1 = 0.20
m.maxPct = 0.80
def Limit_Markdown_Week1_Rule(m,i):
    return m.MarkPct[i,1] <= m.maxPctW1
m.Limit_Markdown_Week1 =  Constraint(m.ITEMS, rule = Limit_Markdown_Week1_Rule)


# Define Constraints

# Define max discount
def Max_Discount_Rule(m,i,w):
    return m.MarkPct[i,w] <= m.maxPct
m.Max_Discount = Constraint(m.ITEMS, m.WEEKS, rule = Max_Discount_Rule)  

# Sell such that leftover inventory is less than min Inv
def Leftover_Inv_Rule(m,i):
     return m.FinalInv[i,4] <= m.cap
m.Leftover_Inv_Rule = Constraint(m.ITEMS, rule = Leftover_Inv_Rule)


# Define Objective Function
def Max_Revenue_Rule(m):
    return sum(m.Revenue[i,w] for i in m.ITEMS for w in m.WEEKS) - sum(m.dispCost*m.FinalInv[i,4] for i in m.ITEMS)
m.Max_Revenue = Objective(rule = Max_Revenue_Rule, sense = maximize)

### Solve

In [3]:
solver = SolverFactory('ipopt')
solution = solver.solve(m)

Check the status of the solver

In [4]:
print(solution['Solver'])


- Status: ok
  Message: Ipopt 3.14.16\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.02962946891784668



### Extract Optimum Values

Print optimum value of objective function

In [5]:
print(m.Max_Revenue.expr())

1755.0400035352961


Print optimum value of decision and implicit variables - Option 2 (generic, same syntax will work for other use cases)

In [6]:
data = {'Variable': [], 'Value': []}
for var in m.component_objects(Var):
    for index in var:
        data['Variable'].append(f'{var.name}[{index}]')
        data['Value'].append(var[index].value)
pd.DataFrame(data)

,Variable,Value
0,"MarkPct[('A', 1)]",2.000000e-01
1,"MarkPct[('A', 2)]",2.666667e-01
2,"MarkPct[('A', 3)]",2.666667e-01
3,"MarkPct[('A', 4)]",2.666667e-01
4,"MarkPct[('B', 1)]",2.000000e-01
5,"MarkPct[('B', 2)]",2.666667e-01
6,"MarkPct[('B', 3)]",2.666667e-01
7,"MarkPct[('B', 4)]",2.666667e-01
8,"MarkPct[('C', 1)]",2.000000e-01
9,"MarkPct[('C', 2)]",2.666667e-01


In [7]:
m.display()

Model unknown

  Variables:
    MarkPct : Size=12, Index=ITEMS*WEEKS
        Key      : Lower : Value               : Upper : Fixed : Stale : Domain
        ('A', 1) :     0 :  0.2000000099125034 :     1 : False : False : NonNegativeReals
        ('A', 2) :     0 :  0.2666666636738239 :     1 : False : False : NonNegativeReals
        ('A', 3) :     0 :  0.2666666636847529 :     1 : False : False : NonNegativeReals
        ('A', 4) :     0 : 0.26666666370661246 :     1 : False : False : NonNegativeReals
        ('B', 1) :     0 : 0.20000000997875883 :     1 : False : False : NonNegativeReals
        ('B', 2) :     0 :  0.2666666634461252 :     1 : False : False : NonNegativeReals
        ('B', 3) :     0 :  0.2666666634487796 :     1 : False : False : NonNegativeReals
        ('B', 4) :     0 : 0.26666666345408857 :     1 : False : False : NonNegativeReals
        ('C', 1) :     0 : 0.20000000998496825 :     1 : False : False : NonNegativeReals
        ('C', 2) :     0 :    0.266666663